<a href="https://colab.research.google.com/github/khanhtran2000/OCR-Dev/blob/master/%08IAM_CRAFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install CRAFT since this is a pretrained module. After installation, the notebook will have to be restarted. 

In [0]:
!pip install craft-text-detection

     |████████████████████████████████| 519.5MB 32kB/s 
     |████████████████████████████████| 25.0MB 1.4MB/s 
     |████████████████████████████████| 25.3MB 1.4MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 2.1MB 41.7MB/s 
     |████████████████████████████████| 31.2MB 148kB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
ERROR: umap-learn 0.4.4 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: fastai 1.0.61 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.
ERROR

# Import modules and connect Colab to Google Drive. 

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import craft
import os
import matplotlib.pyplot as plt
import cv2
import glob
from google.colab.patches import cv2_imshow

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

downloading model
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import labels file.

In [0]:
#Label
df = pd.read_csv('/content/drive/My Drive/IAM word/words_label.txt', delimiter=' ',header=None ,error_bad_lines=False ,engine='python')
df = df.drop(columns=[1,2,3,4,5,6,7]).rename(columns={0: "img_name", 8: "img_label"})
df.head()

Skipping line 16939: ' ' expected after '"'
Skipping line 16941: ' ' expected after '"'
Skipping line 16976: ' ' expected after '"'
Skipping line 63758: field larger than field limit (131072)
Skipping line 23: Expected 9 fields in line 23, saw 10
Skipping line 48: Expected 9 fields in line 48, saw 10
Skipping line 75: Expected 9 fields in line 75, saw 10
Skipping line 100: Expected 9 fields in line 100, saw 10
Skipping line 118: Expected 9 fields in line 118, saw 10
Skipping line 136: Expected 9 fields in line 136, saw 10
Skipping line 196: Expected 9 fields in line 196, saw 10
Skipping line 215: Expected 9 fields in line 215, saw 10
Skipping line 273: Expected 9 fields in line 273, saw 10
Skipping line 291: Expected 9 fields in line 291, saw 10
Skipping line 1345: Expected 9 fields in line 1345, saw 10
Skipping line 1421: Expected 9 fields in line 1421, saw 10
Skipping line 1498: Expected 9 fields in line 1498, saw 10
Skipping line 2351: Expected 9 fields in line 2351, saw 10
Skipping

,img_name,img_label
0,a01-000u-00-00,A
1,a01-000u-00-01,MOVE
2,a01-000u-00-02,to
3,a01-000u-00-03,stop
4,a01-000u-00-04,Mr.


In [0]:
df.head(30)

,img_name,img_label
0,a01-000u-00-00,A
1,a01-000u-00-01,MOVE
2,a01-000u-00-02,to
3,a01-000u-00-03,stop
4,a01-000u-00-04,Mr.
5,a01-000u-00-05,Gaitskell
6,a01-000u-00-06,from
7,a01-000u-01-00,nominating
8,a01-000u-01-01,any
9,a01-000u-01-02,more


In [0]:
df.set_index(df['img_name'], inplace=True)
df = df.drop(['img_name'], axis=1)

# Iterate through all images and convert them to EMNIST format. Finally, output a csv file in EMNIST format (785 columns with the first being character labels in ASCII code).

In [0]:
# Glob the training data and load a single image path

path = "/content/drive/My Drive/IAM word/words/a01/a01-000u/{}.png"

path_len = len(path) - 6

# Two final lists that will be converted into the EMNIST-like dataframe
gray_letters_list = []
letters_label_all = []


for img in df.index.values:

    for file in glob.glob(path.format(img)):
        img = cv2.imread(str(file))
        img_name = str(file)[path_len:-4] 
        bboxes, polys, heatmap = craft.detect_text(img, link_threshold = 99999999, text_threshold = 0.4)
        img_boxed = craft.show_bounding_boxes(img, bboxes)

        # Create a blank dictionary and a list
        letters_dict = {}
        letters_list = []


        for i in range(len(bboxes)):
            x1 = bboxes[i][0][0]
            y1 = bboxes[i][0][1]
            x2 = bboxes[i][1][0]
            y2 = bboxes[i][1][1]
            x3 = bboxes[i][2][0]
            y3 = bboxes[i][2][1]
            x4 = bboxes[i][3][0]
            y4 = bboxes[i][3][1]

            top_left_x = int(min([x1,x2,x3,x4]))
            top_left_y = int(min([y1,y2,y3,y4]))
            bot_right_x = int(max([x1,x2,x3,x4]))
            bot_right_y = int(max([y1,y2,y3,y4]))
            letter = img[top_left_y:bot_right_y, top_left_x:bot_right_x]

            # Append a new key:value pair to the dictionary with top_left_x value as the key
            letters_dict.update({top_left_x:letter})

        # Sort the keys, then append the corresponding values to letters_list
        for i in sorted(letters_dict.keys()):
            letters_list.append(letters_dict[i])

        # If there are as many character images in the list as the length of the label
        if len(letters_list) == len(df.loc[img_name,'img_label']):
            
            # Get the list of character labels. We need to drop the labels of the letters which have 0 in their shape 
            letters_label = list(df.loc[img_name,'img_label'])

            # Save to results folder
            for letter in letters_list:
                
                # Get the shape of the letter array
                letter_shape = letter.shape

                if 0 in letter_shape:
                    # Here we drop that letter from the letters_label list
                    del letters_label[letters_list.index(letter)]
                # If the shape contains 1 dimension equal to 0, cv2.resize() will raise an error
                elif 0 not in letter_shape:

                    letter_edit = np.mean(letter, axis=2)
                    letter_edit = cv2.resize(letter_edit, (28,28))
                    letter_edit = letter_edit / 255.0 # Normalize and convert to float
                    letter_edit = letter_edit.tolist() 

                    pixel_letter = []

                    # Create a list of all pixels 
                    for pixel_list in letter_edit:
                        for pixel in pixel_list:
                            pixel_letter.append(pixel)

                    gray_pixel_letter = []
                    thres_black = 0.25

                    # Change background to black, character to white
                    for pixel in pixel_letter:
                        pixel = 1 - pixel
                        if pixel < thres_black:
                            pixel = 0
                        gray_pixel_letter.append(pixel)

                    def reshape_ima(image):
                        image = image.reshape(28, 28)
                        return image 

                    # Reshape
                    gray_pixel_letter = np.apply_along_axis(reshape_ima, 0, gray_pixel_letter)
                    gray_pixel_letter = gray_pixel_letter * 255
                    gray_pixel_letter = gray_pixel_letter.flatten()

                    gray_letters_list.append(gray_pixel_letter)
                            
                    # Plotting
                    # plt.imshow(gray_pixel_letter, cmap='gray')
                    # plt.show()

            for label in letters_label:
                letters_label_all.append(label)

In [0]:
# Check how many character labels do we have in total
print(len(letters_label_all))

# Convert character to ascii
letters_label_all = [ord(label) for label in letters_label_all]

46


In [0]:
pixels = {}

# Append letters in gray_letters_list (in the flattened format) into pixels dictionary
for i in range(len(letters_label_all)):
    pixels.update({i:gray_letters_list[i]})

# pixels_df has the shape (784,x), .T method is to transpose it to shape (x,784)
# x is the length of letters_label_all that we just checked earlier
pixels_df = pd.DataFrame(pixels).T

# Insert the label column to the first position
pixels_df.insert(0,'label',letters_label_all)

print(pixels_df)

    label           0           1  ...         781         782         783
0      77    0.000000    0.000000  ...    0.000000    0.000000    0.000000
1      79    0.000000    0.000000  ...    0.000000    0.000000    0.000000
2      86    0.000000    0.000000  ...    0.000000    0.000000    0.000000
3      69    0.000000    0.000000  ...    0.000000    0.000000    0.000000
4     116    0.000000    0.000000  ...    0.000000    0.000000    0.000000
5     111  138.000000  156.750000  ...    0.000000    0.000000    0.000000
6     109    0.000000    0.000000  ...    0.000000    0.000000    0.000000
7     111    0.000000    0.000000  ...    0.000000    0.000000    0.000000
8     114    0.000000    0.000000  ...    0.000000    0.000000    0.000000
9     101    0.000000    0.000000  ...   81.026934  122.230953  156.385272
10    105    0.000000    0.000000  ...    0.000000    0.000000    0.000000
11    115    0.000000    0.000000  ...    0.000000    0.000000    0.000000
12    109    0.000000    

In [0]:
# Output csv file
pixels_df.to_csv("/content/drive/My Drive/IAM word/words/a01/results/iam.csv")

In [0]:
#           for filename in os.listdir('/content/result'):
#               files.download('/content/result/'+filename)
